# Projektfeladat
## 1. rész

Betonminőségi vizsgálatokat végeztünk több telephelyen és összegyűjtöttük az adatokat. Mindenkinek elmondtuk, hogy milyen oszlopokat kell készítenie (ezt be is tartották) de sajnos a formátumot és a fájlnevet nem pontosítottuk, így van aki xlsx-et küldött van aki CSV-t.

Mivel várható, hogy még sokszor kapunk így adatot, írj rá egy programot (vagy colab fájlt), ami a következőket el tudja végezni:

- végigmegy a teljes fájlstrukturán megnyitva az összes fájlt
- ha xlsx akkor openpyxl vagy pandas ha csv akkor csv vagy pandas
- olvassuk be az adatokat (egyesítve őket)
- minden adatsorhoz mentsük el azt is, hogy melyik fájlból származik (ha hibás tudjunk szólni az illetőnek)
- ellenőrizzuk is az adatokat Pydantic-el, ha a fájl hibás, fogjuk el a hibát, naplózzuk vagy ignoráljuk és haladjunk tovább.
- tároljuk őket egy adatbázisban vagy pandas dataframe-ben
- dataframe esetén mentsük el Parquett formátumba, sqlite esetén ez megoldott.

Extra:
- ábrázold a cement, age függvényében az erőt (strength) egy scatter diagrammon, hőtérképpel (strength).
- kezeld a gyarkan előforduló kivételeket, hogy a használó értelmes hibaüzeneteket kapjon.


Egy fájl (ha csv) eleje így néz ki:
```CSV
date,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
2025-03-22,141.3,212,0,203.5,0,971.8,748.5,28,29.89
2025-07-04,168.9,42.2,124.3,158.3,10.8,1080.8,796.2,14,23.51
2025-10-14,250,0,95.7,187.4,5.5,956.9,861.2,28,29.22
```

Ha itt a colabban szeretnéd megcsinálni (javasolt, mert itt megvan minden csomag), akkor fel kell töltened ide a zip állományt. Baloldalt mappa ikon (files), vagy futtasd az alább megadott parancsot ami letölti-kitömöríti!

Ha a runtime lekapcsolódik (disconnect) az állományod is elveszik, tehát akkor újra fel kell töltened.

Nyugodtan használj AI segítséget (a colabba be is van építve, alul láthatod is) de azért próbálj meg megérteni minden sort amit létrehozol! (Az nem jó, ha fogalmad nincs róla mit csinál amit futtatsz).

In [ ]:
# futtasd a kódblokkot
!wget -q https://github.com/goteguru/kmooc_example_data/archive/refs/heads/main.zip && unzip -o main.zip && mv kmooc_example_data-main adatok && rm -f main.zip
# <--- bal oldalt a fájlkezelőben (frissítés után) adatok könyvtár

In [ ]:
#
#  Ide jön a te implementációd!
#
#

In [ ]:
# Használd ezt a validációs modellt (min és max értékek a bemenetre):
from datetime import date
from pydantic import BaseModel, Field

class ConcreteExperiment(BaseModel):
    date: date
    cement: float      = Field(ge=0, le=600)
    slag: float        = Field(ge=0, le=400)
    ash: float         = Field(ge=0, le=300)
    water: float       = Field(ge=0, le=400)
    superplastic:float = Field(ge=0, le=50)
    coarseagg:float    = Field(ge=0, le=1500)
    fineagg:float      = Field(ge=0, le=1000)
    age:float          = Field(ge=0, le=365)
    strength:float     = Field(ge=0, le=100)


### 2. rész

Egy állítólag hozzáértő ember azt állította, hogy a beton kötési szilárdság adatokat jól tudnád ábrázolni t-SNE embedding módszerrel hőtérképpel.

Adott hozzá egy minta kódot is ami ezt megteszi (lásd alább). Ez az alábbi kód egy `df` nevű DataFrame-ben várja az adatokat. Ha te nem ilyet használtál, a legegyszerűbb ha átnevezed mielőtt ezt lefuttatod.

- Írd át a kódot, hogy el tudja menteni a képet a mai dátummal!
- Ments ki az eredmény (3 adat minden méréshez) egy CSV vagy XLSX fájlba!

In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import pandas as pd

# ---- Bemeneti változók ----

X_cols = [
    "cement",
    "slag",
    "ash",
    "water",
    "superplastic",
    "coarseagg",
    "fineagg",
    "age",
]

X = df[X_cols].copy()
y = df["strength"].values  # színezéshez

# ---- Standardizáljuk az adatokat ----
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ---- t-SNE 2D ----
tsne = TSNE(
    n_components=2,
    perplexity=30,
    learning_rate="auto",
    init="pca",
    random_state=42,
)

tsne_components = tsne.fit_transform(X_scaled)

df_tsne = pd.DataFrame(
    tsne_components,
    columns=["TSNE1", "TSNE2"]
)
df_tsne["strength"] = y

# ---- 2D "hőtérkép" scatter ----
plt.figure(figsize=(10, 7))

scatter = plt.scatter(
    df_tsne["TSNE1"],
    df_tsne["TSNE2"],
    c=df_tsne["strength"],
    cmap="viridis",
    s=60,
    alpha=0.85,
)

cbar = plt.colorbar(scatter)
cbar.set_label("Strength")

plt.xlabel("t-SNE dim 1")
plt.ylabel("t-SNE dim 2")
plt.title("t-SNE 2D leképezés – Strength hőtérképes színezéssel")
plt.tight_layout()
plt.show()
